Use the pip tool to list installed Python packages and their locations on the system:


In [ ]:
!pip list -v

To install advertools, run the following from the command line:

In [ ]:
!pip install advertools

Check what version of the package you have installed.

In [ ]:
!pip show pandas

In [ ]:
!pip show advertools

Let's use the crawl function of Advertools to browse a website and position the scanned data in a data frame. 

First, we need import the sample data.

In [59]:
import pandas as pd
from advertools import crawl
from advertools import url_to_df

jsonlines is a Python library to simplify working with jsonlines and ndjson data.

This data format is straight-forward: it is simply one valid JSON value per line, encoded using UTF-8. 

JSON is a data interchange format. Files with JSON file extension store data in a human-readable format derived from Javascript programming language.

JSON format is very similar to XML text format and offers high-readability and clear data organization and structure. 

We will use the crawl() function from the advertools library.

Jsonlines is the supported output format because of its flexibility

We have crawled the website and output the data in “jl” format. 

“follow_links=True” attribute is for making crawlers following the URLs on a web page.

Starting with the provided URL(s) go through all links and parse pages.

In [9]:
crawl('https://www.windecor.ca', 'windecor_output_file_Jan112021.jl', follow_links=True)

In [10]:
winpd = pd.read_json('windecor_output_file_Jan112021.jl', lines=True)

2021-01-12 01:05:58,127 | INFO | utils.py:157 | _init_num_threads | NumExpr defaulting to 2 threads.


In [11]:
winpd.head()

,url,url_redirected_to,title,meta_desc,canonical,alt_href,og:locale,og:type,og:title,og:description,og:url,og:site_name,twitter:label1,twitter:data1,jsonld_@context,jsonld_@type,jsonld_@id,jsonld_name,jsonld_priceRange,jsonld_description,jsonld_image,jsonld_url,jsonld_logo,jsonld_telephone,jsonld_areaServed,jsonld_openingHoursSpecification,jsonld_sameAs,jsonld_address.@type,jsonld_address.streetAddress,jsonld_address.addressLocality,jsonld_address.addressRegion,jsonld_address.postalCode,jsonld_address.addressCountry,h1,h2,h3,body_text,size,resp_meta_download_timeout,resp_meta_download_slot,resp_meta_download_latency,resp_meta_depth,status,links_url,links_text,links_fragment,links_nofollow,img_src,img_alt,ip_address,crawl_time,resp_headers_date,resp_headers_content-type,resp_headers_vary,resp_headers_last-modified,resp_headers_cache-control,resp_headers_expires,resp_headers_x-powered-by,resp_headers_cf-cache-status,resp_headers_cf-request-id,resp_headers_expect-ct,resp_headers_report-to,resp_headers_nel,resp_headers_strict-transport-security,resp_headers_x-content-type-options,resp_headers_server,resp_headers_cf-ray,request_headers_accept,request_headers_accept-language,request_headers_user-agent,request_headers_accept-encoding,request_headers_cookie,og:image,og:image:width,og:image:height,twitter:label2,twitter:data2,request_headers_referer,jsonld_@graph,resp_headers_x-frame-options
0,https://www.windecor.ca,https://www.windecor.ca,"Window Coverings - Blinds in Calgary, Airdrie,...",Get FREE professional In-Home consultation tha...,https://www.windecor.ca/,https://www.windecor.ca/feed/@@https://www.win...,en_US,website,"Window Coverings - Blinds in Calgary, Airdrie,...",Get FREE professional In-Home consultation tha...,https://www.windecor.ca/,WinDecor Window Coverings Calgary,Est. reading time,4 minutes,http://schema.org,LocalBusiness,https://www.windecor.ca/#LocalBusiness,WinDecor Window Coverings,$$,"WinDecor Window Coverings provides blinds, sha...",https://www.windecor.ca/wp-content/uploads/201...,https://www.windecor.ca,https://windecor.ca/wp-content/uploads/2013/04...,+1 -403-775-9097,"[https://en.wikipedia.org/wiki/Calgary, https:...","[{'@type': 'OpeningHoursSpecification', 'dayOf...",[https://www.houzz.com/pro/rahulmehrotra/winde...,PostalAddress,9266 Santana Crescent NW,Calgary,Alberta,T3K3N9,CA,Call for Window Coverings in Calgary:@@\n@@\n,\nWinDecor provides beautiful blinds and shade...,Schedule FREE Appointment,(403) 775-9097 \n Virtual Consultation Sch...,91105,180,www.windecor.ca,0.749587,0,200,https://www.windecor.ca/@@https://www.windecor...,@@Virtual Consultation@@Schedule FREE Appointm...,@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@...,False@@False@@False@@False@@False@@False@@Fals...,"data:image/svg+xml,%3Csvg%20xmlns='http://www....",Phone Number@@Phone Number@@@@@@@@@@Window Tre...,172.67.73.143,2021-01-12 01:05:46,"Tue, 12 Jan 2021 01:05:46 GMT",text/html; charset=UTF-8,"Accept-Encoding,User-Agent","Mon, 11 Jan 2021 15:26:21 GMT",max-age=0,"Tue, 12 Jan 2021 01:05:46 GMT",PleskLin,DYNAMIC,0795b9b1a700001857c3137000000001,"max-age=604800, report-uri=""https://report-uri...","{""endpoints"":[{""url"":""https:\/\/a.nel.cloudfla...","{""report_to"":""cf-nel"",""max_age"":604800}",max-age=15552000,nosniff,cloudflare,6102f895db841857-ATL,"text/html,application/xhtml+xml,application/xm...",en,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"gzip, deflate",__cfduid=d2f87cd54f1f886edb083f1d52e860cf51610...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.windecor.ca/hunter-douglas-motoriz...,https://www.windecor.ca/hunter-douglas-motoriz...,Hunter Douglas Motorized Blinds - FAQ - Part 1...,Hunter Douglas Motorized Blinds - What are the...,https://www.windecor.ca/hunter-douglas-motoriz...,https://www.windecor.ca/feed/@@https://www.win...,en_US,article,Hunter Douglas Motorized Blinds - FAQ - Part 1...,Hunter Douglas Motorized Blinds - What are the...,https://www.windecor.ca/hunter-douglas-motoriz...,WinDecor Wind

In [ ]:
winpd.shape

In [ ]:
winpd.columns

Are there missing titles?

In [ ]:
winpd.title.isna().value_counts()

False    93
Name: title, dtype: int64

Good, we don't have any missing title tag

In [ ]:
winpd['title']

Are there any missing meta description?

In [ ]:
winpd.meta_desc.isna().value_counts()

False    51
True     42
Name: meta_desc, dtype: int64

42 missing description in this case, where are those??

In [ ]:
winpd[winpd['meta_desc'].isna()]

In [ ]:
winpd.loc[winpd.meta_desc.isnull()]

Let's check Status Codes of URLs

In [ ]:
winpd.status.value_counts()

In [ ]:
winpd.shape

The 93 URL's in the dataframe have Status = 200

How to check Average Meta Title Length for our website pages?

In [ ]:
winpd.title.str.len().mean()

In [ ]:
winpd.title.str.len().describe()

How Many Title Tags are Longer than 60 Characters?

In [ ]:
winpd.title.str.len().value_counts()

In [ ]:
winpd.loc[winpd.title.str.len() > 60]

In [ ]:
type(winpd.loc[winpd.title.str.len() > 60])

pandas.core.frame.DataFrame

In [ ]:
type((winpd.title.str.len() > 60))

pandas.core.series.Series

In [ ]:
(winpd.title.str.len() > 60).value_counts()

True     60
False    33
Name: title, dtype: int64

60 of our meta titles are longer than 60 characters. Is this GOOD or BAD??

Let's see — Meta Titles Consisting of Certain Number of Words - How to explore?

In [13]:
winpd.title[0]

'Window Coverings - Blinds in Calgary, Airdrie, Okotoks, Cochrane'

In [2]:
import re
s_marks = 'Window Coverings - Blinds in Calgary, Airdrie, Okotoks, Cochrane'
#results = re.split('[-\s,]', s_marks)
res = list(filter(None, re.split('[-\s,]', s_marks)))
print(res)
len(res)

['Window', 'Coverings', 'Blinds', 'in', 'Calgary', 'Airdrie', 'Okotoks', 'Cochrane']


8

In [78]:
winpd.title[92]

'What blinds will look elegant in your home | FREE Consultation | WinDecor'

In [3]:
import re
p_marks = 'What blinds will look elegant in your home | FREE Consultation | WinDecor'
#results = re.split('[-\s,|]', p_marks)
results = list(filter(None, re.split('[-\s,|]', p_marks)))
print(results)
len(results)

['What', 'blinds', 'will', 'look', 'elegant', 'in', 'your', 'home', 'FREE', 'Consultation', 'WinDecor']


11

In [ ]:
winpd['title_word_count'] = [len(list(filter(None, re.split('[-\s,|]', title)))) for title in winpd.title]

winpd.head(5)

In [ ]:
pd.set_option('display.max_colwidth',155)
winpd.filter(['title_word_count', 'title']).head(10)

In [ ]:
(winpd.title_word_count == 0).value_counts()

In [48]:
winpd.title_word_count.mean()

8.89247311827957

In [ ]:
winpd.title_word_count.describe()

In [ ]:
winpd[winpd['title'].str.len() > 60].url.to_frame()

How to Categorize URLs of a Web Site?

In [63]:
category_pd = url_to_df(winpd['url_redirected_to'], decode=True)
category_pd.tail(20)

,url,scheme,netloc,path,query,fragment,dir_1,dir_2,dir_3
73,https://www.windecor.ca/products/shades/concept-dual-roller-shades/,https,www.windecor.ca,/products/shades/concept-dual-roller-shades/,,,products,shades,concept-dual-roller-shades
74,https://www.windecor.ca/products/blinds/aluminum-blinds/,https,www.windecor.ca,/products/blinds/aluminum-blinds/,,,products,blinds,aluminum-blinds
75,https://www.windecor.ca/tag/faux-wood-blinds/,https,www.windecor.ca,/tag/faux-wood-blinds/,,,tag,faux-wood-blinds,NaN
76,https://www.windecor.ca/products/blinds/wood-blinds/,https,www.windecor.ca,/products/blinds/wood-blinds/,,,products,blinds,wood-blinds
77,https://www.windecor.ca/products/shades/roller-shades/,https,www.windecor.ca,/products/shades/roller-shades/,,,products,shades,roller-shades
78,https://www.windecor.ca/products/blinds/vinyl-blinds/,https,www.windecor.ca,/products/blinds/vinyl-blinds/,,,products,blinds,vinyl-blinds
79,https://www.windecor.ca/tag/motorized-shades/,https,www.windecor.ca,/tag/motorized-shades/,,,tag,motorized-shades,NaN
80,https://www.windecor.ca/tag/motorized-blinds/,https,www.windecor.ca,/tag/motorized-blinds/,,,tag,motorized-blinds,NaN
81,https://www.windecor.ca/products/blinds/faux-wood-blinds/,https,www.windecor.ca,/products/blinds/faux-wood-blinds/,,,products,blinds,faux-wood-blinds
82,https://www.windecor.ca/tag/motorized-window-coverings/,https,www.windecor.ca,/tag/motorized-window-coverings/,,,tag,motorized-window-coverings,NaN


In [62]:
category_pd.shape

(93, 9)

In [72]:
category_pd[category_pd.dir_1 == 'zebra-blinds']

,url,scheme,netloc,path,query,fragment,dir_1,dir_2,dir_3
13,https://www.windecor.ca/zebra-blinds/,https,www.windecor.ca,/zebra-blinds/,,,zebra-blinds,NaN,NaN


Which category has how many URLs?

In [67]:
category_pd.groupby('dir_1').url.count().sort_values(ascending=False)

dir_1
tag                                                              34
products                                                         23
portfolio                                                        10
blog                                                              3
cordless-blinds-kids-safety                                       2
schedule-free-consultation                                        2
dual-shades-room-darkening-shade                                  2
zebra-blinds                                                      1
about-us                                                          1
are-zebra-blinds-good-for-calgary-homes                           1
banded-sheer-roller-shades-check-these-3-facts-before-you-buy     1
blinds-and-shades-element-of-decoration                           1
cdn-cgi                                                           1
contact-blinds-shades                                             1
faux-wood-vs-real-wood-blinds             

In [ ]:
category_pd.groupby('dir_1').apply(lambda category_pd: category_pd.url.iloc[0])

In [70]:
category_pd.groupby(['dir_1', 'dir_2']).url.count()

dir_1      dir_2                             
blog       page                                   1
cdn-cgi    l                                      1
portfolio  50-off-motorized-blinds                1
           choose-your-perfect-shades             1
           dont-miss-out-cellular-blinds-sale     1
           free-cordless-control                  2
           free-cordless-roller-roman-blinds      1
           honeycomb-blinds-rebate                1
           save-big-on-blinds                     1
           save-on-motorized-blinds               1
           shutters-gear-system-free-upgrade      1
products   blinds                                 7
           motorized-blinds-and-shades            1
           shades                                11
           shutters                               2
           window-curtains-drapes                 1
tag        automated-blinds                       1
           banded-shades                          1
           bedroom

In [73]:
category_pd.groupby(['dir_1', 'dir_2']).apply(lambda category_pd: category_pd.url.iloc[0])

dir_1      dir_2                             
blog       page                                                                   https://www.windecor.ca/blog/page/2/
cdn-cgi    l                                                        https://www.windecor.ca/cdn-cgi/l/email-protection
portfolio  50-off-motorized-blinds                          https://www.windecor.ca/portfolio/50-off-motorized-blinds/
           choose-your-perfect-shades                    https://www.windecor.ca/portfolio/choose-your-perfect-shades/
           dont-miss-out-cellular-blinds-sale    https://www.windecor.ca/portfolio/dont-miss-out-cellular-blinds-sale/
           free-cordless-control                               https://www.windecor.ca/portfolio/free-cordless-control
           free-cordless-roller-roman-blinds      https://www.windecor.ca/portfolio/free-cordless-roller-roman-blinds/
           honeycomb-blinds-rebate                          https://www.windecor.ca/portfolio/honeycomb-blinds-rebate/
  